In [1]:
from datetime import timedelta
import pandas as pd
import itertools

In [35]:
def calculate_price_variation(cotizacion, fecha, dias):
    # Asegúrate de que la columna 'Date' es de tipo datetime
    cotizacion['Date'] = pd.to_datetime(cotizacion['Date'])

    # Define el rango de dias
    n_dias = timedelta(days=dias)

    # Filtra las filas por el rango de fechas
    filtered_rows = cotizacion.loc[(cotizacion['Date'] >= fecha - n_dias) & 
                                          (cotizacion['Date'] <= fecha)].copy()

    # Verifica si hay datos disponibles
    if filtered_rows.empty:
        return None, None

    # Calcula la variación en porcentaje entre el precio de apertura y cierre
    filtered_rows['Variation'] = ((filtered_rows['Close'] - filtered_rows['Open']) / filtered_rows['Open']) * 100

    # Agrega el valor de 'Open', 'Close' y la diferencia entre ellos en la fecha de interés
    news_day_open = filtered_rows.loc[filtered_rows['Date'] == fecha, 'Open'].values
    news_day_close = filtered_rows.loc[filtered_rows['Date'] == fecha, 'Close'].values

    # Verifica si hay datos disponibles para la fecha de interés
    if news_day_open.size == 0 or news_day_close.size == 0:
        return None, None

    # Determina si el precio subió, bajó o se mantuvo igual
    if news_day_close[0] > news_day_open[0]:
        price_movement = "subió"
    elif news_day_close[0] < news_day_open[0]:
        price_movement = "bajó"
    else:
        price_movement = "se mantuvo igual"

    # Devuelve la variación en porcentaje y el movimiento del precio
    return filtered_rows['Variation'].tolist(), price_movement


def analyze_price_movement(filtered_gpt, cotizacion_Endesa, dias):
    # Supongamos que filtered_gpt es tu DataFrame con las fechas almacenadas
    filtered_gpt_dates = filtered_gpt['Date']

    # Contadores para el número de veces que ha subido, bajado o se ha mantenido igual
    count_up = 0
    count_down = 0
    count_same = 0
    total_variation_up = 0
    total_variation_down = 0

    # Iterar sobre las fechas y calcular la variación para cada una
    for date_of_interest in filtered_gpt_dates:
        date_of_interest = pd.to_datetime(date_of_interest)
        price_variation_list, price_movement = calculate_price_variation(cotizacion_Endesa, date_of_interest, dias)

        # Analizar el movimiento del precio y actualizar los contadores
        if price_movement == "subió":
            count_up += 1
            total_variation_up += sum(price_variation_list) if price_variation_list else 0
        elif price_movement == "bajó":
            count_down += 1
            total_variation_down += sum(price_variation_list) if price_variation_list else 0
        else:
            count_same += 1

    # Calcular las probabilidades
    total_days_with_movement = count_up + count_down + count_same

    # Evitar la división por cero
    prob_up = count_up / total_days_with_movement * 100 if total_days_with_movement > 0 else 0
    prob_down = count_down / total_days_with_movement * 100 if total_days_with_movement > 0 else 0
    prob_same = count_same / total_days_with_movement * 100 if total_days_with_movement > 0 else 0

    # Calcular la media de variación cuando las acciones suben
    mean_variation_up = total_variation_up / count_up if count_up > 0 else None

    # Calcular la media de variación cuando las acciones bajan
    mean_variation_down = total_variation_down / count_down if count_down > 0 else None

    # Imprimir el resultado
    if mean_variation_up is not None:
        print(f"Hará subir las acciones un {prob_up:.2f}% de las veces en los próximos {dias} días desde que se dio la noticia. Media de variación en subidas: {mean_variation_up:.2f}%")
    else:
        print(f"Hará subir las acciones un {prob_up:.2f}% de las veces en los próximos {dias} días desde que se dio la noticia. Media de variación en subidas: No disponible")
    if mean_variation_down is not None:
        print(f"Hará bajar las acciones un {prob_down:.2f}% de las veces en los próximos {dias} días desde que se dio la noticia. Media de variación en bajadas: {mean_variation_down:.2f}%")
    else:
        print(f"Hará bajar las acciones un {prob_down:.2f}% de las veces en los próximos {dias} días desde que se dio la noticia. Media de variación en bajadas: No disponible")
    print(f"El precio se mantendrá igual un {prob_same:.2f}% de las veces en los próximos {dias} días desde que se dio la noticia.")
    print("*" * 137)
            
        
def analyze_news_effect(filtered_gpt, cotizacion, dias_list=[1, 2, 5, 7]):
    if not filtered_gpt.empty:
        sentido = "Positiva" if filtered_gpt['sentido'].iloc[0] > 0 else "Negativa"
        print("Una noticia del tipo", filtered_gpt['tipo'].iloc[0], sentido ,"de", filtered_gpt['empresa'].iloc[0],":\n")
        print("*" * 137)
        for dias in dias_list:
            analyze_price_movement(filtered_gpt, cotizacion, dias)
    else:
        print("No hay datos para analizar.")

In [37]:

# Leemos el fichero que contiene las salidas de GPT
gpt = pd.read_csv('./data/gpt.csv', index_col=0)

# Preparamos los tipos de noticias y empresas para filtrar el DataFrame 'gpt'
# empresas = ["Endesa", "Iberdrola", "Solaria", "Naturgy", "Acciona_Energias"]
empresas = ["Endesa"]
tipo_noticias = ["Negocios y Finanzas Corporativas", "Regulación y Asuntos Legales", "Desarrollo de Infraestructura y Proyectos", "Dividendos", "Estrategia y Planificación Empresarial"]
sentido = [0.9, -0.5]

# Conjunto para realizar un seguimiento de las combinaciones únicas
combinaciones_procesadas = set()

result_df = []

# Iteramos sobre todas las combinaciones posibles una vez
for empresa, tipo, sent in itertools.product(empresas, tipo_noticias, sentido):
    # Verificamos si esta combinación ya ha sido procesada
    if (empresa, tipo, sent) in combinaciones_procesadas:
        continue

    # Filtra el DataFrame 'gpt' 
    filtered_gpt = gpt.query(f'empresa == "{empresa}" and tipo == "{tipo}" and sentido > {sent}')

    # Elimina la columna 'prefijo' del DataFrame 'filtered_gpt'
    filtered_gpt = filtered_gpt.drop(columns=['prefijo'])

    # Convierte la columna 'Date' a datetime
    filtered_gpt['Date'] = pd.to_datetime(filtered_gpt['Date'])

    # Elimina la parte de la hora de la columna 'Date'
    filtered_gpt['Date'] = filtered_gpt['Date'].dt.date

    # Leemos el fichero con los datos
    cotizacion = pd.read_csv('./data/cotizacion_' + empresa + '.csv')

    # Realiza la busqueda
    analyze_news_effect(filtered_gpt, cotizacion)


Una noticia del tipo Negocios y Finanzas Corporativas Positiva de Endesa :

*****************************************************************************************************************************************
Hará subir las acciones un 46.55% de las veces en los próximos 1 días desde que se dio la noticia. Media de variación en subidas: 1.17%
Hará bajar las acciones un 40.52% de las veces en los próximos 1 días desde que se dio la noticia. Media de variación en bajadas: -0.98%
El precio se mantendrá igual un 12.93% de las veces en los próximos 1 días desde que se dio la noticia.
*****************************************************************************************************************************************
Hará subir las acciones un 46.55% de las veces en los próximos 2 días desde que se dio la noticia. Media de variación en subidas: 1.02%
Hará bajar las acciones un 40.52% de las veces en los próximos 2 días desde que se dio la noticia. Media de variación en bajadas: -0.85